# FHIR Terminology Services
## REST Server API

In [1]:
!pip -q install requests
!pip -q install jsonasobj

In [2]:
import sys
from typing import Optional, Union, Dict
import requests
from jsonasobj import JsonObj, loads, as_json
import getpass


class FHIR:
    """ FHIR server wrapper """
    def __init__(self, url: str,
                 user: Optional[str] = None,
                 password: Optional[str] = None,
                 default_format: str = "json",
                 show_urls: bool = False) -> None:

        self.url = url + ('' if url.endswith('/') or url.endswith('#') else '/')
        self.format = default_format
        self._user = user
        self._password = password
        self.metadata = self.get('metadata')
        self.ok = self.metadata is not None
        self.show_urls = show_urls

    @staticmethod
    def logon(url: str, show_urls: bool = False) -> Optional["FHIR"]:
        """ Prompt for a user and password and return a server """
        server = None
        while server is None:
            user = input(f"User id: ")
            if not user:
                return None
            else:
                password = getpass.getpass(f"Password: ")
            server = FHIR(url, user, password, show_urls=show_urls)
        return server

    def __getitem__(self, item: str) -> "FHIRReader":
        """ Convert item to a URI """
        return FHIRReader(item, self)

    def __getattr__(self, item: str) -> Union[object, Optional[JsonObj]]:
        if item.startswith('_'):
            return super().__getattribute__(item)
        else:
            return FHIRReader(item, self)

    def get(self, relurl: Optional[str], params: Optional[Dict[str, str]] = None) -> Optional[JsonObj]:
        url = self.url + relurl
        if self.show_urls:
            print(f"GET {url}")
        resp = requests.get(url, params=params, auth=(self._user, self._password) if self._user else None)
        if resp.ok:
            return loads(resp.text)
        else:
            print(f"{url}: Error: {resp.reason}", file=sys.stderr)
            return None


class FHIRReader:
    def __init__(self, resource: str, server: FHIR) -> None:
        """
        FHIR Resource reader
        :param resource: resource type
        :param server: FHIR server
        """
        self._resource = resource
        self._server = server

    def __getitem__(self, item: str) -> Optional[JsonObj]:
        """ Return <resource>[item] """
        return self._server.get(self._resource + ('/' + item if item != '*' else ''))

    def __getattr__(self, item: str) -> Union[object, Optional[JsonObj]]:
        """ Return <resource>.<item> """
        return super().__getattribute__(item) if item.startswith('_') else self.__getitem__(item)
    
    def lookup(self, **kwargs) -> Optional[JsonObj]:
        return self._server.get(self._resource + '/$lookup', params=kwargs)


## Connect to the server

In [3]:
default_url = 'https://fhir.loinc.org/'
server = FHIR.logon(default_url, show_urls=True)

if server is None:
    sys.exit(1)

User id: hsolbrig
Password: ········
GET https://fhir.loinc.org/metadata


## Server metadata

In [4]:
def print_metadata(server: FHIR) -> None:
    m = server.metadata
    print(f"Server {m.implementation.description} at {m.implementation.url} is based on {m.software.name}({m.software.version})")
    print("Resources:")
    supported_resources = []
    for rest in m.rest:
        if rest.mode == 'server':
            for r in rest.resource:
                if hasattr(r, 'interaction'):
                    capabilities = \
                        sorted(list({c.code[0].upper() if c.code in ('create', 'read', 'update', 'delete', 'search') else 'O'
                                    for c in r.interaction}))
                else:
                    capabilities = []
                supported_resources.append(r.type)
                print(f"\t{r.type}: {''.join(capabilities)}")
    print()

    if 'CodeSystem' in supported_resources:
        print("CodeSystems")
        for e in server.CodeSystem['*'].entry:
            print(f"\t{e.resource.name}: {e.resource.title if 'title' in e.resource else 'MISSING Title'} - {e.resource.description if 'description' in e.resource else 'MISSING Description'} ")
        print()

    if 'ValueSet' in supported_resources:
        print("ValueSets")
        for e in server.ValueSet['*'].entry:
            print(f"\t{e.resource.id}: {e.resource.name}")
        print()
print_metadata(server)

Server FHIR REST Server at https://fhir.loinc.org is based on Smile CDR(2019.08.PRE)
Resources:
	CodeSystem: CDORU
	ConceptMap: CDORU
	OperationDefinition: R
	StructureDefinition: OR
	Subscription: 
	ValueSet: CDORU

CodeSystems
GET https://fhir.loinc.org/CodeSystem
	LOINC: LOINC Code System - LOINC is a freely available international standard for tests, measurements, and observations 

ValueSets
GET https://fhir.loinc.org/ValueSet
	LL4136-9: Anticoagulant medications
	LL4137-7: Preoperative procedures
	LL415-1: MERS_TH_15_Eviron-safety event
	LL4152-6: CMS IRFPAI_GG0110 Prior Device Use
	LL1922-5: NEMSIS_75_EMS condition codes
	LL4153-4: CMS IRFPAI_GG Functional Abilies
	LL2040-5: Body Surface Area Methods-DICOM
	LL4154-2: CMS IRFPAI_GG Functional Goals
	LL2110-6: ESRD-open-closed status
	LL4155-9: CMS IRFPAI_H0350 Bladder Continence
	LL2153-6: VA-36-work/daily activity limitations
	LL4752-3: NTDS_I_14_Protective devices
	LL2154-4: VA-36-work/social activity limitations
	LG37183-7: Al

## Look up concept codes

In [5]:
code = None
while code != '':
    code=input("Enter concept code: ")
    if code:
        loinc_code = server.CodeSystem.lookup(system="http://loinc.org", code=code)
        if loinc_code:
            display = ''.join(p.valueString for p in loinc_code.parameter if p.name == "display")
            print(f"{code}: {display}")
        else:
            print(f"Code: {code} not found")

Enter concept code: 4548-4
GET https://fhir.loinc.org/CodeSystem/$lookup
4548-4: Hemoglobin A1c/Hemoglobin.total in Blood
Enter concept code: 217042-1


https://fhir.loinc.org/CodeSystem/$lookup: Error: Not Found


GET https://fhir.loinc.org/CodeSystem/$lookup
Code: 217042-1 not found
Enter concept code: 27042-1
GET https://fhir.loinc.org/CodeSystem/$lookup
27042-1: Methohexital [Mass/volume] in Serum or Plasma
Enter concept code: 


# Switch to a different server

In [6]:
server = FHIR("https://r4.ontoserver.csiro.au/fhir", show_urls=True)
print_metadata(server)


GET https://r4.ontoserver.csiro.au/fhir/metadata
Server Ontoserver, The Australian e-Health Research Centre, CSIRO at https://r4.ontoserver.csiro.au/fhir is based on Ontoserver®(6.0.0-SNAPSHOT)
Resources:
	Account: 
	ActivityDefinition: 
	AdverseEvent: 
	AllergyIntolerance: 
	Appointment: 
	AppointmentResponse: 
	AuditEvent: C
	Basic: 
	BiologicallyDerivedProduct: 
	BodyStructure: 
	Bundle: CDORU
	CapabilityStatement: 
	CarePlan: 
	CareTeam: 
	CatalogEntry: 
	ChargeItem: 
	ChargeItemDefinition: 
	Claim: 
	ClaimResponse: 
	ClinicalImpression: 
	CodeSystem: CDORU
	Communication: 
	CommunicationRequest: 
	CompartmentDefinition: 
	Composition: 
	ConceptMap: CDORU
	Condition: 
	Consent: 
	Contract: 
	Coverage: 
	CoverageEligibilityRequest: 
	CoverageEligibilityResponse: 
	DetectedIssue: 
	Device: 
	DeviceDefinition: 
	DeviceMetric: 
	DeviceRequest: 
	DeviceUseStatement: 
	DiagnosticReport: 
	DocumentManifest: 
	DocumentReference: 
	Encounter: 
	Endpoint: 
	EnrollmentRequest: 
	EnrollmentRes

	v3-ActClassProcedure: v3.ActClassProcedure
	v2-0918: v2.0918
	v2-0617: v2.0617
	verificationresult-can-push-updates: can-push-updates
	medication-form-1.0.1: medication-form
	v2-0761: v2.0761
	procedure-act-status-hl7-v3-1.0.0: procedure-act-status-hl7-v3-1
	v3-ActClassDocument: v3.ActClassDocument
	v3-ProcessingID: v3.ProcessingID
	v2-0492: v2.0492
	expression-language: ExpressionLanguage
	v2-0572: v2.0572
	v2-0569: v2.0569
	v3-hl7PublishingDomain: v3.hl7PublishingDomain
	orientation-type: orientationType
	variants: HGVS
	v2-0163: v2.0163
	medication-status: Medication Status Codes
	related-artifact-type: RelatedArtifactType
	v2-0554: v2.0554
	v3-EquipmentAlertLevel: v3.EquipmentAlertLevel
	example-filter: ACMECholCodesPlasma
	map-context-type: StructureMapContextType
	v2-0426: v2.0426
	v2-0920: v2.0920
	encounter-reason: EncounterReasonCodes
	list-empty-reason: ListEmptyReasons
	medication-admin-status: MedicationAdministration Status Codes
	v3-policyHolderRole: v3.policyHolderRole
